# Resting state features calculations

In [1]:
# imports
from nice import Markers
from nice.markers import (PowerSpectralDensity,
                          PowerSpectralDensitySummary,
                          PowerSpectralDensityEstimator,
                          KolmogorovComplexity,
                          PermutationEntropy,
                          SymbolicMutualInformation)
import winsound
import time

In [1]:
# load basic libraries
import pandas as pd
import numpy as np
import mne
from pathlib import Path
import pickle
import time

from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score

%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

# set directories
# %cd D:/Programy/Anaconda3/Projects/EEG ML project # working directory
%cd D:
pkls = './Pickles/' # objects & variables

D:\Programy\Anaconda3\Projects\EEG ML project


In [3]:
# custom functions for markers
from scipy.stats import trim_mean
def mean_trim80(a, axis=0):  
    return trim_mean(a, proportiontocut=.1, axis=axis)
def entropy(a, axis=0): 
    return -np.nansum(a * np.log(a), axis=axis) / np.log(a.shape[axis])

### Prepare markers

In [7]:
backend = 'python'  # maximum compatibility across platforms

# define one base estimator to avoid recomputation later on
# when making another set of marker, also recompute the base_psd
psds_params = dict(n_fft = 384,     # length of the FFT used (if None: the FFT length is nperseg)
                   n_overlap = 192, # n of points to overlap between segments (if None: then nperseg/2)
                   n_jobs = 'auto', # cpu cores
                   nperseg = 4096)  # length of each segment (samples)

base_psd = PowerSpectralDensityEstimator(
    psd_method='welch', tmin=None, tmax=None, fmin=1., fmax=45.,
    psd_params=psds_params, comment='default')


# resting-state compatible markers, gamma band removed
markers = Markers([
    PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=4.,
                         normalize=False, comment='delta'),
    PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=4.,
                         normalize=True, comment='deltan'),
    PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8.,
                         normalize=False, comment='theta'),
    PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8.,
                         normalize=True, comment='thetan'),
    PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=12.,
                         normalize=False, comment='alpha'),
    PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=12.,
                         normalize=True, comment='alphan'),
    PowerSpectralDensity(estimator=base_psd, fmin=12., fmax=30.,
                         normalize=False, comment='beta'),
    PowerSpectralDensity(estimator=base_psd, fmin=12., fmax=30.,
                         normalize=True, comment='betan'),
    PowerSpectralDensity(estimator=base_psd, fmin=30., fmax=45.,
                         normalize=False, comment='gamma'),
    PowerSpectralDensity(estimator=base_psd, fmin=30., fmax=45.,
                         normalize=True, comment='gamman'),
    
    PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=45.,
                         normalize=False, comment='summary_se'),
    PowerSpectralDensitySummary(estimator=base_psd, fmin=1., fmax=45.,  # aka Spectral Entropy
                                percentile=.5, comment='summary_msf'),
    PowerSpectralDensitySummary(estimator=base_psd, fmin=1., fmax=45.,
                                percentile=.9, comment='summary_sef90'),
    PowerSpectralDensitySummary(estimator=base_psd, fmin=1., fmax=45.,
                                percentile=.95, comment='summary_sef95'),
    PermutationEntropy(tmin=None, tmax=0.6, backend=backend),
    SymbolicMutualInformation(
        tmin=None, tmax=0.6, method='weighted', backend=backend,
        method_params={'nthreads': 'auto', 'bypass_csd': True}, # csd needs to be skipped
        comment='weighted'),
    KolmogorovComplexity(tmin=None, tmax=0.6, backend=backend,
                         method_params={'nthreads': 'auto'}),
])

In [21]:
# # optional: compute markers before reducing them (explore and plot)
# markers.fit(epochs)
# markers.save('fullmarkers1.hdf5', overwrite = True)

# # optional: explore PSDs used for the marker computation
# psd = base_psd.data_
# freqs = base_psd.freqs_

#%matplotlib widget
# plt.figure()
# plt.semilogy(freqs, np.mean(psd, axis=0).T, alpha=0.1, color='black')
# plt.xlim(2, 40)
# plt.ylabel('log(psd)')
# plt.xlabel('Frequency [Hz]')
# plt.show()

NameError: name 'epochs' is not defined

### Set data files 

In [139]:
# local computer
%cd D:
path = Path('D:/Programy/Anaconda3/Projects/EEG ML project/Epochs clean/')
epo_list = list(path.glob('*.fif'))
len(epo_list)

D:\Programy\Anaconda3\Projects\EEG ML project


9

In [7]:
# remote computer
%cd D:
path = Path('D:/Marcin/OneDrive - Uniwersytet Jagielloński/Nauka/Projekty/2020.02 - Bartek ML EEG/Epochs clean/')
epo_list = list(path.glob('*.fif'))
len(epo_list)

D:\Bartek\JupyterLab Projects\ML EEG project


10

### Prepare and compute reductions

In [ ]:
# set timer
start_time = time.time()

# settings
nparticipants     = 2                     # n of participants to append
epochs_all_funs   = [mean_trim80, np.std] # reduction functions for epochs
channels_all_funs = [np.mean, np.std]     # reduction functions for channels

# column names
nice_columns = ['d','d_norm','t','t_norm','a','a_norm','b','b_norm','g', 'g_norm',
                'summ_se','summ_msf','summ_sef90','summ_sef95','PE', 'wSMI', 'K']

# nested loops
r3 = pd.DataFrame() # empty df for appending participnts
loopstep_n = 0

# iterate over participants
# for file in epo_list[:nparticipants]:
for file in (epo_list[:6] + epo_list[10:14]): # for testing
    
    epochs = mne.read_epochs(fname = file) # read epoched eeg data
    x_features = np.empty((len(epochs), len(markers))) # empty array for reductions
    r2 = pd.DataFrame(columns = ['epoch','id']) # empty df for appending participnts
    
    # iterate over function
    for epochs_fun in epochs_all_funs:
        for channels_fun in channels_all_funs:

            loopstep_n += 1
            print('\n - Loop step %i -'     % loopstep_n)
            print('Epochs   function: %s'   % epochs_fun.__name__)
            print('Channels function: %s\n' % channels_fun.__name__)

            # set reduction parameters
            reduction_params = {
                'PowerSpectralDensity': {
                    'reduction_func': [
                        {'axis': 'frequency', 'function': np.sum},
                        {'axis': 'epochs', 'function': epochs_fun},
                        {'axis': 'channels', 'function': channels_fun}]
                },
                'PowerSpectralDensitySummary': {
                    'reduction_func': [
                        {'axis': 'epochs', 'function': epochs_fun},
                        {'axis': 'channels', 'function': channels_fun}]
                },
                'SymbolicMutualInformation': {
                    'reduction_func': [
                        {'axis': 'epochs', 'function': epochs_fun},
                        {'axis': 'channels', 'function': channels_fun},
                        {'axis': 'channels_y', 'function': channels_fun}]
                },
                'PermutationEntropy': {
                    'reduction_func': [
                        {'axis': 'epochs', 'function': epochs_fun},
                        {'axis': 'channels', 'function': channels_fun}]
                },
                'KolmogorovComplexity': {
                    'reduction_func': [
                        {'axis': 'epochs', 'function': epochs_fun},
                        {'axis': 'channels', 'function': channels_fun}]
                }
            }

            # compute markers over epochs
            for ii in range(len(epochs)):
                markers.fit(epochs[ii])
                x_features[ii, :] = markers.reduce_to_scalar(marker_params=reduction_params) # reduce
                for marker in markers.values():
                    delattr(marker, 'data_')
                delattr(base_psd, 'data_')
        
            # merge reduction functions
            r1 = pd.DataFrame.from_records(x_features) # to df
            
            if file.stem[:5] == 'video':
                r1.insert(0, 'id', file.stem[6:11]) # add id (video session) 
            else:
                r1.insert(0, 'id', file.stem[:5]) # add id (open/closed eyes)
                        
            r1 = r1.rename(columns=dict(zip(r1.columns[1:], nice_columns)))
            r1.reset_index(inplace = True)
            r1.rename(columns={'index' : 'epoch'}, inplace = True) 
            r1.columns = r1.columns.map(lambda x: x + ('_' + epochs_fun.__name__[0] + channels_fun.__name__[0]) \
                                        if x != 'id' and x != 'epoch' else x) # suffix functions abbreviations
            r2 = r2.merge(r1, how = 'right', on = ['id', 'epoch'])
        
    # append participants
    r3 = r3.append(r2)
    r3.reset_index(inplace = True, drop = True)    
    
# get execution time
print("--- %s seconds ---" % round(time.time() - start_time, 2))
winsound.Beep(frequency = 440, duration = 800) # beep

### Save etc

In [90]:
# save
with open('features_r3.pkl', 'wb') as handle:
    pickle.dump(r3, handle)

In [15]:
# load
# %cd D:
with open(pkls +'r3.pkl', 'rb') as handle:
    r3 = pickle.load(handle)

D:\Programy\Anaconda3\Projects\EEG ML project
